In [2]:
!pip install sparsesvd

In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import pandas as pd
import numpy as np
import random
import sys
import time
import evaluation
import dataprocess
import utilities
from collections import OrderedDict
from scipy import sparse
from sparsesvd import sparsesvd

topN = [5, 10, 15, 20]

d = "d1"
p = "explicit"

benchmark = ["Amazon"]#,["Amazon"]# "ML100k ML1M", "Ciao", "Watcha"]
hiddenDim = [5, 10, 15, 20]
hyperParams = OrderedDict()
res = []
temp = []

with tf.Graph().as_default():
    # define top-K
    prediction_top_k = tf.placeholder(tf.float32, [None, None])
    scale_top_k = tf.placeholder(tf.int32)
    top_k = tf.nn.top_k(prediction_top_k, scale_top_k)

    config = tf.ConfigProto(device_count={'GPU': 0})
    sess = tf.Session(config=config)
    sess.run(tf.global_variables_initializer())
    
    for be in benchmark:
        userCount, itemCount, trainSet, adjList_user, adjList_item, testMaskArray = dataprocess.loadTrainDictionary(be, d, p)
        testSet = dataprocess.loadTestData(be, d)

        userList_test = list(testSet.keys())
        userList_train = list(trainSet.keys())
        batchSize_test = 500

        for h in hiddenDim:
            Precision, Recall, NDCG, MRR = evaluation.prepareMetrics(topN, be)
            # prepare sparse matrix of training data
            sparse_rating_matrix = sparse.lil_matrix((userCount, itemCount))
            for u, i_list in trainSet.items():
                for j in i_list:
                    if p == "explicit":
                        sparse_rating_matrix[u, j[0]] = j[1]
                    else:
                        sparse_rating_matrix[u, j] = 1.0

            P, Sigma, Q = sparsesvd(sparse.csc_matrix(sparse_rating_matrix), h)
            predictedIndices = []

            numOfMinibatches_validation = int(len(userList_train) / batchSize_test) + 1
            numOfLastMinibatch_validation = len(userList_train) % batchSize_test

            t1 = time.time()
            for batchId in range(numOfMinibatches_validation):
                batchUserIndex = []
                start = batchId * batchSize_test
                if batchId == numOfMinibatches_validation-1:   #if it is the last minibatch
                    numOfBatches = numOfLastMinibatch_validation
                else:
                    numOfBatches = batchSize_test
                end = start + numOfBatches

                batchMask = []
                _prediction = []
                
                for i in range(start, end):
                    # this user
                    userId = userList_train[i]

                    # prediction of this user
                    R = np.dot(P.T[userId], np.dot(np.diag(Sigma), Q))
                    _prediction.append(R)
                    

                _prediction = np.array(_prediction)
                _prediction = _prediction + testMaskArray[start: end]
#                 print(batchId, end=' ')
#                 np.save('./'+str(h)+'/'+str(batchId)+'_saved', _prediction)

                _, predict_minibatch = sess.run(top_k, feed_dict={prediction_top_k: _prediction, scale_top_k: topN[-1]})
                predictedIndices.extend(predict_minibatch)
            
            df = pd.DataFrame(predictedIndices)
            df.to_csv(str(h)+'_top_result.csv', sep=' ')
            display(df)
            Precision, Recall, NDCG, MRR = evaluation.computeTopNAccuracy (testSet, userList_test, predictedIndices, topN, Precision, Recall, NDCG, MRR)
            t2 = time.time()
            hyperParams['hiddenDim'] = h
            # report results on files
            utilities.printBaselineResults(be, topN, "pureSVD", [hyperParams], Precision, Recall, NDCG, MRR, str(t2-t1))
            

    sess.close()


print("finish!!")


Instructions for updating:
non-resource variables are not supported in the long term
datasets/Amazon/d1.train
loading TrainDictionary done....
288106
26491
sorting TrainDictionary done....
datasets/Amazon/d1.test
loading test set done....


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,16844,17031,16456,21440,15287,15013,18753,18939,15908,16483,19300,3216,2299,1070,1907,15632,25921,14581,25186,12977
1,16844,21440,17031,15013,18753,15287,15908,16483,18939,2299,14581,25921,15632,16456,3216,23681,12977,19300,16971,26126
2,16844,16456,15908,17031,21440,15287,15013,18939,18753,16483,2299,3216,19300,14581,25921,1070,15632,23681,1907,7816
3,15908,16456,17031,21440,15287,15013,18939,18753,16483,2299,3216,19300,14581,1070,25921,15632,23681,1907,12977,7816
4,16844,21440,17031,15013,18753,15287,16483,18939,15908,2299,14581,25921,15632,3216,23681,19300,12977,16971,26126,7816
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288101,15908,16844,21440,17031,18939,15013,18753,500,14581,15287,23681,20135,16483,2299,6485,1070,3216,25558,25921,12379
288102,15908,16844,21440,17031,18939,15013,18753,500,14581,15287,23681,20135,16483,2299,6485,1070,3216,25558,25921,12379
288103,15908,16844,21440,17031,18939,15013,18753,500,14581,15287,23681,20135,16483,2299,6485,1070,3216,25558,25921,12379
288104,15908,16844,21440,17031,18939,15013,18753,500,14581,15287,23681,20135,16483,2299,6485,1070,3216,25558,25921,12379


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,3216,19300,2736,16483,1907,17031,2299,4606,15013,18753,7838,16537,3370,25186,6262,3334,4404,23786,11761,7816
1,3216,19300,2736,16483,17031,21440,2299,1907,18753,4606,7816,7838,3370,25186,16537,4404,6262,14581,23786,11761
2,3216,19300,2736,16483,21440,1907,18753,2299,17031,4606,16456,7838,16537,3370,25186,7816,4404,11761,6262,23786
3,3216,19300,2736,16483,1907,17031,2299,4606,18753,7838,16537,3370,7816,25186,11761,6262,4404,23786,8363,3334
4,3216,19300,2736,16483,17031,2299,1907,15013,4606,7816,7838,16537,3370,25186,6262,14581,15287,3334,4404,11761
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288101,15908,3216,2736,19300,16483,17031,2299,1907,4606,7816,7838,16537,14581,11761,15287,25558,3370,23681,6262,25186
288102,15908,3216,2736,19300,16483,17031,2299,1907,4606,7816,7838,16537,14581,11761,15287,25558,3370,23681,6262,25186
288103,15908,3216,2736,19300,16483,17031,2299,1907,4606,7816,7838,16537,14581,11761,15287,25558,3370,23681,6262,25186
288104,15908,3216,2736,19300,16483,17031,2299,1907,4606,7816,7838,16537,14581,11761,15287,25558,3370,23681,6262,25186


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,3216,19300,1907,2299,6262,7838,18753,3334,25186,25558,17038,5790,19283,13369,14581,7517,2865,19502,2984,23786
1,3216,19300,1907,2299,21440,18753,4606,2736,7816,7838,11761,6262,25186,16537,14581,25558,3370,3334,17031,23681
2,3216,19300,16483,1907,2299,18753,21440,7838,16456,6262,25186,3334,25558,11761,4606,3370,23786,17038,14581,16537
3,3216,7816,19300,4606,1907,2299,11761,7838,16537,18753,13153,2736,3370,6262,25186,3334,8363,25558,23786,7414
4,3216,19300,1907,2299,16483,17031,7838,15013,14581,6262,25558,23681,3334,25186,17038,6485,5790,16971,19283,13369
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288101,15908,3216,19300,1907,16844,2299,7838,11761,4606,7816,14581,25558,6262,23681,16537,25186,6485,3334,2865,3370
288102,15908,3216,19300,1907,16844,2299,7838,11761,4606,7816,14581,25558,6262,23681,16537,25186,6485,3334,2865,3370
288103,15908,3216,19300,1907,16844,2299,7838,11761,4606,7816,14581,25558,6262,23681,16537,25186,6485,3334,2865,3370
288104,15908,3216,19300,1907,16844,2299,7838,11761,4606,7816,14581,25558,6262,23681,16537,25186,6485,3334,2865,3370


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,3216,19300,1907,2299,3334,6262,17038,25186,16627,5790,13369,25558,18753,19502,295,7517,19283,2984,14720,2865
1,19300,3216,4606,1907,2299,18753,21440,7838,11761,16537,2736,3370,6262,7816,25186,17031,14581,4404,25558,23786
2,3216,16483,1907,4606,18753,2299,7609,7838,21440,19300,11761,4404,16537,16456,6262,25186,3370,25558,23786,3334
3,4606,11761,7838,16537,1907,4404,3370,18753,2299,8363,6262,23786,7414,25186,25558,5379,14581,2856,19283,3334
4,19300,3216,1907,2299,17031,4606,16483,7838,6262,14581,15013,25558,11761,23681,25186,3334,4404,3370,16537,6485
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288101,15908,4606,11761,3216,7838,1907,2299,16537,16844,25558,14581,6262,3370,23681,6485,8363,25186,19283,7414,23786
288102,15908,4606,11761,3216,7838,1907,2299,16537,16844,25558,14581,6262,3370,23681,6485,8363,25186,19283,7414,23786
288103,15908,4606,11761,3216,7838,1907,2299,16537,16844,25558,14581,6262,3370,23681,6485,8363,25186,19283,7414,23786
288104,15908,4606,11761,3216,7838,1907,2299,16537,16844,25558,14581,6262,3370,23681,6485,8363,25186,19283,7414,23786


finish!!
